In [ ]:
!pip install --upgrade openai

import json
import time
import random
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key="")  # Replace with your real key

output_file = "unique_normal_funny_dataset.jsonl2"
num_pairs = 5000  # or however many you want

seen_sentences = set()

adjectives = [
    "beautiful", "big", "small", "cute", "funny", "colorful", "bright", "dark", "noisy", "quiet",
    "fast", "slow", "happy", "sad", "old", "young", "warm", "cold", "soft", "hard",
    "friendly", "lazy", "strong", "weak", "clean", "dirty", "fresh", "spicy", "sweet", "sour",
    "bitter", "smooth", "rough", "shiny", "dull", "loud", "calm", "peaceful", "exciting", "boring",
    "busy", "empty", "tasty", "healthy", "dangerous", "safe", "messy", "neat", "rich", "poor",
    "simple", "complex", "modern", "traditional", "rare", "common", "new", "old-fashioned",
    "energetic", "relaxed", "bright", "dark", "warm", "cool", "gentle", "harsh", "bold", "shy",
    "brave", "timid", "strong", "fragile", "friendly", "hostile"
]

# Diverse prompts to encourage different outputs
topics = [
    "dogs", "cats", "fishing", "cooking", "gardening", "reading", "biking", "hiking", "running", 
    "coffee", "tea", "weather", "TV shows", "movies", "music", "video games", "camping", "sports", 
    "yoga", "fitness", "gym", "traveling", "vacation", "weekend", "school", "college", "homework", 
    "work", "meetings", "commute", "traffic", "cars", "trains", "planes", "grocery shopping", 
    "laundry", "cleaning", "cooking dinner", "breakfast", "lunch", "snacks", "pets", "birds", 
    "hamsters", "lizards", "aquariums", "beaches", "mountains", "forests", "rivers", "sunsets", 
    "sunrises", "stars", "clouds", "rain", "snow", "wind", "thunderstorms", "clothes", "shoes", 
    "fashion", "style", "shopping", "sales", "deals", "birthdays", "holidays", "Christmas", 
    "Thanksgiving", "Halloween", "New Year's", "Easter", "Valentine's Day", "weekends", "Mondays", 
    "Fridays", "sleep", "dreams", "waking up", "coffee shops", "fast food", "restaurants", "dessert", 
    "ice cream", "cake", "cookies", "fruit", "vegetables", "meat", "BBQ", "pizza", "burgers", 
    "pasta", "noodles", "sushi", "music festivals", "concerts", "books", "novels", "comics", 
    "poetry", "writing", "drawing", "painting", "photography", "technology", "phones", "apps", 
    "internet", "Wi-Fi", "social media", "Instagram", "Twitter", "TikTok", "YouTube", "Facebook", 
    "texting", "calling", "email", "zoom", "FaceTime", "family", "friends", "neighbors", 
    "roommates", "siblings", "parents", "grandparents", "kids", "babies", "toddlers", 
    "school supplies", "laptops", "notebooks", "pens", "pencils", "shopping carts", "mall", 
    "parking", "dogs barking", "cats meowing", "birds chirping", "alarm clocks", "candles", 
    "lighting", "home decor", "plants", "succulents", "flowers", "vases", "tools", "DIY", 
    "renovation", "moving", "packing", "unpacking", "recycling", "trash", "chores", "budgets", 
    "money", "banking", "saving", "credit cards", "paychecks", "paying bills", "rent", "mortgage", 
    "housing", "apartments", "houses", "furniture", "TV", "sofa", "bed", "sheets", "pillows", 
    "blankets", "showers", "bathrooms", "toilets", "kitchen", "fridge", "microwave", "oven", 
    "stove", "dishes", "silverware", "cups", "mugs", "sports games", "football", "soccer", 
    "basketball", "baseball", "tennis", "golf", "swimming", "skating", "skiing", "snowboarding", 
    "surfing", "kayaking", "canoeing", "rollerblading", "skateboarding", "board games", 
    "card games", "chess", "puzzles", "lego", "toys", "cartoons", "anime", "superheroes", "zoo", 
    "museum", "library", "park", "playground", "picnic", "grilling", "fireworks", "bonfire", 
    "campfire", "marshmallows", "s'mores", "backyard", "front yard", "neighbors", "mail", 
    "packages", "delivery", "Amazon", "online shopping", "returns", "receipts", "customer service", 
    "appointments", "dentist", "doctor", "hospital", "pharmacy", "medicine", "vitamins", "health", 
    "hydration", "water bottles", "coffee mugs", "thermos", "bags", "backpacks", "purses", "wallets", 
    "keys", "phones", "chargers", "headphones", "earbuds", "Bluetooth", "speakers", "music playlists", 
    "alarm", "reminders", "calendars", "notes", "journal", "diary", "goals", "habits", 
    "resolutions", "motivation", "productivity", "morning routine", "evening routine", 
    "sleep schedule", "relaxation", "meditation", "deep breathing", "stretching"
]


with open(output_file, "w", encoding="utf-8") as f:
    i = 0
    attempts = 0
    while i < num_pairs and attempts < num_pairs * 2:
        try:
            topic = random.choice(topics)
            adjective = random.choice(adjectives)
            prompt = f"Write one simple, everyday, neutral sentence about {adjective} {topic}."

            # Step 1: Generate neutral sentence
            response_normal = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"{prompt} (Sentence #{i+1})"}
                ],
                max_tokens=30,
                temperature=0.7,
            )
            normal_sentence = response_normal.choices[0].message.content.strip()

            # Skip duplicates
            if normal_sentence in seen_sentences:
                print(f"Duplicate detected: '{normal_sentence}' — Skipping.")
                attempts += 1
                continue

            seen_sentences.add(normal_sentence)

            # Step 2: Make it funny
            funny_prompt = f"Make this sentence funny: {normal_sentence}"
            response_funny = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a witty comedian."},
                    {"role": "user", "content": funny_prompt}
                ],
                max_tokens=60,
                temperature=0.9,
            )
            funny_sentence = response_funny.choices[0].message.content.strip()

            # Save result in JSONL format
            data = {
                "input": f"Make this sentence funny: {normal_sentence}",
                "target": funny_sentence
            }
            f.write(json.dumps(data, ensure_ascii=False) + "\n")

            print(f"{i+1}/{num_pairs} | Input: {normal_sentence}")
            print(f"             Funny: {funny_sentence}\n")

            i += 1
            time.sleep(1)

        except Exception as e:
            print(f"Error at instance {i+1}: {e}")
            time.sleep(5)